In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from synorchestrator import orchestrator
from synorchestrator.trs.client import TRSClient
from synorchestrator.wes.client import WESClient
from synorchestrator import config

In [3]:
config.show()


Orchestrator options:

Evaluation Queues
---------------------------------------------------------------------------
wflow0: github.com/dockstore-testing/md5sum-checker [CWL]
wflow1: github.com/dockstore-testing/md5sum-checker/wdl [WDL]
wflow2: github.com/DataBiosphere/topmed-workflows/TopMed_Variant_Caller [WDL]
wflow3: github.com/DataBiosphere/topmed-workflows/u_of_Michigan_alignment_pipeline [WDL]

Tool Registries
---------------------------------------------------------------------------
dockstore: dockstore.org:8443

Workflow Services
---------------------------------------------------------------------------
hca-cromwell: g0n2qjnu94.execute-api.us-east-1.amazonaws.com/test
arvados-wes: wes.qr1hi.arvadosapi.com
local: 0.0.0.0:8080


In [52]:
# orchestrator.run_all({
#     'wflow2': ['hca-cromwell'],
#     'wflow3': ['hca-cromwell']
# })
submissions = orchestrator.run_all({
    'wflow0': ['arvados-wes']
})

INFO:synorchestrator.orchestrator:Preparing checker workflow run request for 'github.com/dockstore-testing/md5sum-checker' from  'dockstore''
INFO:synorchestrator.eval:Created new job submission:
 - evaluation queue: wflow0 (github.com/dockstore-testing/md5sum-checker)
 - submission ID: '250525203328505703'
 - data:
{
  "workflow_descriptor": "cwlVersion: v1.0\nclass: Workflow\n\nrequirements:\n  - class: SubworkflowFeatureRequirement\n\n#dct:creator:\n#  '@id': http://orcid.org/0000-0002-7681-6415\n#  foaf:name: Brian O'Connor\n#  foaf:mbox: mailto:briandoconnor@gmail.com\n\n#dct:contributor:\n#  foaf:name: Denis Yuen\n#  foaf:mbox: mailto:denis.yuen@oicr.on.ca\n\ninputs:\n  input_file: File\n  expected_md5: string\n\noutputs:\n  workflow_output_file:\n    type: File\n    outputSource: checker/results_file\n\nsteps:\n  md5sum:\n    run: md5sum/md5sum-workflow.cwl\n    in:\n      input_file: input_file\n    out: [output_file]\n  checker:\n    run: checker/md5sum-checker.cwl\n    in:\n 

In [53]:
orchestrator.monitor(submissions)

,,submission_status,elapsed_time,job,wes_id,queue_id,run_status,run_id,start_time
md5sum-checker,250525203328505703,SUBMITTED,0h:0m:8s,checker,arvados-wes,md5sum-checker,CANCELED,qr1hi-xvhdp-yi4vt14cdrb0y3i,Fri May 25 20:33:29 2018


Done
